# Quantitative Crypto Research - Orderbook Feature Analysis

## Research Context

**Researcher**: Senior Quant - Crypto Markets  
**Goal**: Extract alpha from high-frequency orderbook microstructure  
**Asset**: BTC-USD on Coinbase Advanced  
**Data**: 197 engineered features from L2 orderbook snapshots + trades

### Research Questions

1. **Can orderbook microstructure predict short-term price direction?**
2. **Which features carry the most predictive signal?**
3. **What are the optimal prediction horizons for our feature frequency?**
4. **How do different ML models compare on this high-frequency data?**

### Key References
- Cont et al. (2014): Order Flow Imbalance (OFI) predicts price changes
- Kyle (1985): Lambda measures market impact
- López de Prado (2018): Financial ML best practices

---

## 1. Setup & Data Loading

In [ ]:
# Core imports
import os
import sys
import warnings
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import polars as pl

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ML
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report,
    confusion_matrix, mean_squared_error, r2_score
)

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

print(f"Research session started: {datetime.now()}")
print(f"Polars version: {pl.__version__}")

In [ ]:
# ============================================================================
# DATA CONFIGURATION
# ============================================================================

# Paths
DATA_ROOT = Path("data/processed/silver/orderbook")
EXCHANGE = "coinbaseadvanced"
SYMBOL = "BTC-USD"

# For initial exploration, load a sample (1 day)
# For production modeling, use full scan_parquet
SAMPLE_MODE = True  # Set False for full data

if SAMPLE_MODE:
    # Load specific partition for fast iteration
    sample_path = DATA_ROOT / f"exchange={EXCHANGE}/symbol={SYMBOL}/year=2026/month=1/day=26"
    print(f"Sample mode: Loading from {sample_path}")
    df = pl.read_parquet(f"{sample_path}/**/*.parquet")
else:
    # Full lazy scan for production
    full_path = DATA_ROOT / f"exchange={EXCHANGE}/symbol={SYMBOL}/**/*.parquet"
    print(f"Full mode: Scanning {full_path}")
    df = pl.scan_parquet(str(full_path)).collect()

print(f"\nLoaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory: {df.estimated_size('mb'):.1f} MB")

In [ ]:
# ============================================================================
# FEATURE CATEGORIZATION
# ============================================================================

# Define feature groups for analysis
FEATURE_GROUPS = {
    'meta': ['timestamp', 'capture_ts', 'symbol', 'exchange', 'year', 'month', 'day', 'hour'],
    'price_levels': [c for c in df.columns if 'price_L' in c],
    'size_levels': [c for c in df.columns if 'size_L' in c and 'bid' in c or 'ask' in c],
    'core_prices': ['best_bid', 'best_ask', 'mid_price', 'spread', 'relative_spread', 'microprice'],
    'imbalances': [c for c in df.columns if 'imbalance' in c.lower()],
    'depth': [c for c in df.columns if 'depth' in c.lower() or 'total_' in c],
    'ofi': [c for c in df.columns if 'ofi' in c.lower() or 'mlofi' in c.lower()],
    'dynamics': [c for c in df.columns if any(x in c for x in ['velocity', 'acceleration', 'log_return'])],
    'rolling': [c for c in df.columns if 'rolling' in c.lower() or 'realized_vol' in c or 'mean_' in c],
    'advanced': [c for c in df.columns if any(x in c for x in ['kyle', 'vpin', 'toxicity', 'lambda'])],
    'liquidity': [c for c in df.columns if any(x in c for x in ['vwap', 'smart', 'concentration', 'slope'])],
}

# Print summary
print("Feature Group Summary:")
print("=" * 50)
total_categorized = 0
for group, cols in FEATURE_GROUPS.items():
    existing = [c for c in cols if c in df.columns]
    print(f"{group:15s}: {len(existing):3d} features")
    total_categorized += len(existing)

# Find uncategorized
all_categorized = set()
for cols in FEATURE_GROUPS.values():
    all_categorized.update(cols)
uncategorized = [c for c in df.columns if c not in all_categorized]
print(f"{'uncategorized':15s}: {len(uncategorized):3d} features")
print(f"\nTotal columns: {len(df.columns)}")

## 2. Exploratory Data Analysis

In [ ]:
# ============================================================================
# TEMPORAL ANALYSIS
# ============================================================================

# Check timestamp distribution
df = df.sort('timestamp')

# Calculate inter-arrival times
df = df.with_columns([
    (pl.col('timestamp').diff().dt.total_milliseconds() / 1000).alias('delta_seconds')
])

print("Timestamp Analysis:")
print(f"  Start: {df['timestamp'].min()}")
print(f"  End:   {df['timestamp'].max()}")
print(f"  Duration: {(df['timestamp'].max() - df['timestamp'].min())}")

print(f"\nInter-arrival time (seconds):")
delta_stats = df.select('delta_seconds').drop_nulls().describe()
print(delta_stats)

In [ ]:
# ============================================================================
# PRICE DYNAMICS VISUALIZATION
# ============================================================================

# Sample for plotting (every 100th row for large datasets)
sample_rate = max(1, len(df) // 5000)
df_plot = df.gather_every(sample_rate).to_pandas()

fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('Mid Price', 'Spread (bps)', 'Order Flow Imbalance', 'L1 Imbalance'),
    row_heights=[0.3, 0.2, 0.25, 0.25]
)

# Mid price
fig.add_trace(
    go.Scatter(x=df_plot['timestamp'], y=df_plot['mid_price'], 
               name='Mid Price', line=dict(color='blue', width=1)),
    row=1, col=1
)

# Spread in bps
if 'relative_spread' in df_plot.columns:
    spread_bps = df_plot['relative_spread'] * 10000
    fig.add_trace(
        go.Scatter(x=df_plot['timestamp'], y=spread_bps,
                   name='Spread (bps)', line=dict(color='orange', width=1)),
        row=2, col=1
    )

# OFI
if 'ofi' in df_plot.columns:
    fig.add_trace(
        go.Scatter(x=df_plot['timestamp'], y=df_plot['ofi'],
                   name='OFI', line=dict(color='green', width=1)),
        row=3, col=1
    )

# L1 Imbalance
if 'imbalance_L1' in df_plot.columns:
    colors = ['red' if x < 0 else 'green' for x in df_plot['imbalance_L1']]
    fig.add_trace(
        go.Bar(x=df_plot['timestamp'], y=df_plot['imbalance_L1'],
               name='L1 Imbalance', marker_color=colors),
        row=4, col=1
    )

fig.update_layout(
    height=800,
    title_text=f"{SYMBOL} Orderbook Microstructure",
    showlegend=False
)
fig.show()

In [ ]:
# ============================================================================
# FEATURE DISTRIBUTIONS
# ============================================================================

# Select key features for distribution analysis
key_features = [
    'relative_spread', 'imbalance_L1', 'ofi', 'microprice',
    'smart_depth_imbalance', 'total_depth_20'
]
key_features = [f for f in key_features if f in df.columns]

if key_features:
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.flatten()
    
    for i, feat in enumerate(key_features[:6]):
        data = df[feat].drop_nulls().to_numpy()
        
        # Clip outliers for visualization
        q01, q99 = np.percentile(data, [1, 99])
        data_clipped = np.clip(data, q01, q99)
        
        axes[i].hist(data_clipped, bins=50, edgecolor='black', alpha=0.7)
        axes[i].set_title(f'{feat}\nmean={np.mean(data):.4f}, std={np.std(data):.4f}')
        axes[i].axvline(np.mean(data), color='red', linestyle='--', label='mean')
    
    plt.tight_layout()
    plt.suptitle('Key Feature Distributions', y=1.02, fontsize=14)
    plt.show()

## 3. Target Engineering

We create multiple prediction targets at different horizons to understand which timeframes our features can predict.

In [ ]:
# ============================================================================
# TARGET VARIABLE CREATION
# ============================================================================

# Define prediction horizons (in number of ticks/rows)
# Adjust based on your data frequency
HORIZONS = {
    '1tick': 1,
    '5tick': 5,
    '10tick': 10,
    '30tick': 30,
    '60tick': 60,
}

# Create forward returns and direction targets
target_exprs = []
for name, shift in HORIZONS.items():
    # Forward log return
    target_exprs.append(
        (pl.col('mid_price').shift(-shift) / pl.col('mid_price')).log().alias(f'ret_{name}')
    )
    # Binary direction (1 = up, 0 = down)
    target_exprs.append(
        (pl.col('mid_price').shift(-shift) > pl.col('mid_price')).cast(pl.Int8).alias(f'dir_{name}')
    )

df = df.with_columns(target_exprs)

# Verify target creation
print("Target Variables Created:")
print("=" * 50)
for name in HORIZONS.keys():
    ret_col = f'ret_{name}'
    dir_col = f'dir_{name}'
    
    ret_stats = df.select(ret_col).drop_nulls()
    dir_stats = df.select(dir_col).drop_nulls()
    
    up_pct = dir_stats[dir_col].sum() / len(dir_stats) * 100
    
    print(f"\n{name}:")
    print(f"  Return: mean={ret_stats[ret_col].mean():.6f}, std={ret_stats[ret_col].std():.6f}")
    print(f"  Direction: {up_pct:.1f}% up, {100-up_pct:.1f}% down")

In [ ]:
# ============================================================================
# TARGET AUTOCORRELATION ANALYSIS
# ============================================================================
# Check if returns are autocorrelated (mean-reverting or trending)

from scipy import stats

print("Return Autocorrelation Analysis:")
print("=" * 50)

for name in HORIZONS.keys():
    ret_col = f'ret_{name}'
    returns = df[ret_col].drop_nulls().to_numpy()
    
    # Lag-1 autocorrelation
    if len(returns) > 100:
        autocorr_1 = np.corrcoef(returns[:-1], returns[1:])[0, 1]
        autocorr_5 = np.corrcoef(returns[:-5], returns[5:])[0, 1]
        
        print(f"\n{ret_col}:")
        print(f"  Lag-1 autocorr: {autocorr_1:.4f}")
        print(f"  Lag-5 autocorr: {autocorr_5:.4f}")
        
        # Interpretation
        if autocorr_1 < -0.05:
            print(f"  → Mean-reverting (negative autocorr)")
        elif autocorr_1 > 0.05:
            print(f"  → Trending (positive autocorr)")
        else:
            print(f"  → Random walk (near-zero autocorr)")

## 4. Feature-Target Correlation Analysis

Identify which features have predictive power for our targets.

In [ ]:
# ============================================================================
# FEATURE-TARGET CORRELATIONS
# ============================================================================

# Select numeric features (exclude meta, targets)
meta_cols = FEATURE_GROUPS['meta']
target_cols = [c for c in df.columns if c.startswith('ret_') or c.startswith('dir_')]
exclude_cols = meta_cols + target_cols + ['delta_seconds']

feature_cols = [
    c for c in df.columns 
    if c not in exclude_cols 
    and df[c].dtype in [pl.Float64, pl.Float32, pl.Int64, pl.Int32]
]

print(f"Analyzing {len(feature_cols)} numeric features against targets...")

# Convert to pandas for correlation analysis
df_corr = df.select(feature_cols + ['ret_5tick', 'dir_5tick']).drop_nulls().to_pandas()

# Calculate correlations with 5-tick return (our primary target)
correlations = {}
for col in feature_cols:
    if col in df_corr.columns:
        corr = df_corr[col].corr(df_corr['ret_5tick'])
        correlations[col] = corr

# Sort by absolute correlation
corr_sorted = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

print("\nTop 20 Features by |Correlation| with ret_5tick:")
print("=" * 60)
for feat, corr in corr_sorted[:20]:
    bar = '█' * int(abs(corr) * 50)
    sign = '+' if corr > 0 else '-'
    print(f"{feat:40s} {sign}{abs(corr):.4f} {bar}")

In [ ]:
# ============================================================================
# CORRELATION HEATMAP FOR TOP FEATURES
# ============================================================================

# Select top correlated features
top_features = [f for f, _ in corr_sorted[:15]]
top_features_with_target = top_features + ['ret_5tick']

# Correlation matrix
corr_matrix = df_corr[top_features_with_target].corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(
    corr_matrix, 
    annot=True, 
    fmt='.2f',
    cmap='RdBu_r',
    center=0,
    ax=ax
)
plt.title('Feature Correlation Matrix (Top 15 + Target)')
plt.tight_layout()
plt.show()

## 5. ML Model Training with AutoGluon

We use AutoGluon's TabularPredictor for automated model selection and ensembling.

In [ ]:
# ============================================================================
# AUTOGLUON SETUP
# ============================================================================

try:
    from autogluon.tabular import TabularPredictor
    AUTOGLUON_AVAILABLE = True
    print("AutoGluon available ✓")
except ImportError:
    AUTOGLUON_AVAILABLE = False
    print("AutoGluon not installed. Install with: pip install autogluon")
    print("Falling back to sklearn models...")

In [ ]:
# ============================================================================
# DATA PREPARATION FOR MODELING
# ============================================================================

# Use top correlated features + some domain knowledge features
model_features = list(set(
    top_features + 
    ['ofi', 'imbalance_L1', 'relative_spread', 'mid_price', 'microprice']
))
model_features = [f for f in model_features if f in df.columns]

# Target
TARGET = 'dir_5tick'  # Binary classification

# Prepare dataset
df_model = df.select(model_features + [TARGET, 'timestamp']).drop_nulls()

# Time-based train/test split (80/20)
# IMPORTANT: For time series, we split by time, not random
n_total = len(df_model)
n_train = int(n_total * 0.8)

train_df = df_model[:n_train].to_pandas()
test_df = df_model[n_train:].to_pandas()

# Remove timestamp for modeling
train_df = train_df.drop(columns=['timestamp'])
test_df = test_df.drop(columns=['timestamp'])

print(f"Training samples: {len(train_df):,}")
print(f"Test samples: {len(test_df):,}")
print(f"Features: {len(model_features)}")
print(f"Target: {TARGET}")
print(f"\nClass balance (train): {train_df[TARGET].value_counts().to_dict()}")

In [ ]:
# ============================================================================
# TRAIN AUTOGLUON MODEL
# ============================================================================

if AUTOGLUON_AVAILABLE:
    # Model save path
    MODEL_PATH = f"models/autogluon_{SYMBOL.replace('-', '_')}_{TARGET}"
    
    # Create predictor
    predictor = TabularPredictor(
        label=TARGET,
        path=MODEL_PATH,
        problem_type='binary',
        eval_metric='roc_auc',  # Optimize for AUC
    )
    
    # Train with time limit (adjust based on your hardware)
    # presets: 'best_quality', 'high_quality', 'good_quality', 'medium_quality'
    predictor.fit(
        train_data=train_df,
        presets='medium_quality',  # Start fast, increase for production
        time_limit=300,  # 5 minutes max
        verbosity=2,
    )
    
    print("\nTraining complete!")
else:
    print("Skipping AutoGluon training (not installed)")

In [ ]:
# ============================================================================
# MODEL EVALUATION
# ============================================================================

if AUTOGLUON_AVAILABLE and 'predictor' in dir():
    # Leaderboard
    print("Model Leaderboard:")
    print("=" * 80)
    leaderboard = predictor.leaderboard(test_df, silent=True)
    print(leaderboard.to_string())
    
    # Predictions
    y_pred = predictor.predict(test_df)
    y_pred_proba = predictor.predict_proba(test_df)
    y_true = test_df[TARGET]
    
    # Metrics
    print("\n" + "=" * 80)
    print("Test Set Performance:")
    print("=" * 80)
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"ROC-AUC:  {roc_auc_score(y_true, y_pred_proba[1]):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Down', 'Up']))

In [ ]:
# ============================================================================
# FEATURE IMPORTANCE
# ============================================================================

if AUTOGLUON_AVAILABLE and 'predictor' in dir():
    print("Computing feature importance (permutation-based)...")
    
    importance = predictor.feature_importance(test_df)
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    importance_sorted = importance.sort_values('importance', ascending=True)
    
    ax.barh(importance_sorted.index, importance_sorted['importance'])
    ax.set_xlabel('Permutation Importance')
    ax.set_title(f'Feature Importance for {TARGET} Prediction')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(importance.head(10))

## 6. Sklearn Fallback Models

If AutoGluon is not available, we use standard sklearn models.

In [ ]:
# ============================================================================
# SKLEARN MODELS (FALLBACK)
# ============================================================================

if not AUTOGLUON_AVAILABLE:
    from sklearn.ensemble import (
        RandomForestClassifier, 
        GradientBoostingClassifier,
        HistGradientBoostingClassifier
    )
    from sklearn.linear_model import LogisticRegression
    
    # Prepare data
    X_train = train_df.drop(columns=[TARGET])
    y_train = train_df[TARGET]
    X_test = test_df.drop(columns=[TARGET])
    y_test = test_df[TARGET]
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Models to try
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=100, n_jobs=-1),
        'Hist Gradient Boosting': HistGradientBoostingClassifier(max_iter=100),
    }
    
    results = []
    for name, model in models.items():
        print(f"\nTraining {name}...")
        
        # Use scaled data for linear models
        if 'Logistic' in name:
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            y_proba = model.predict_proba(X_test_scaled)[:, 1]
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_proba = model.predict_proba(X_test)[:, 1]
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_proba)
        
        results.append({'Model': name, 'Accuracy': acc, 'AUC': auc})
        print(f"  Accuracy: {acc:.4f}, AUC: {auc:.4f}")
    
    # Results summary
    results_df = pd.DataFrame(results).sort_values('AUC', ascending=False)
    print("\n" + "=" * 50)
    print("Model Comparison:")
    print(results_df.to_string(index=False))

## 7. Strategy Backtest Preview

Quick simulation of a trading strategy based on model predictions.

In [ ]:
# ============================================================================
# SIMPLE STRATEGY BACKTEST
# ============================================================================

# Get predictions (use AutoGluon if available, else best sklearn model)
if AUTOGLUON_AVAILABLE and 'predictor' in dir():
    predictions_proba = predictor.predict_proba(test_df)[1].values
else:
    # Use the best sklearn model
    best_model = models['Hist Gradient Boosting']
    predictions_proba = best_model.predict_proba(X_test)[:, 1]

# Prepare backtest data
backtest_df = test_df.copy()
backtest_df['pred_proba'] = predictions_proba
backtest_df['signal'] = (backtest_df['pred_proba'] > 0.5).astype(int) * 2 - 1  # -1 or +1

# Calculate actual returns for the horizon
# Note: This uses 5-tick forward return which may need adjustment
backtest_df['actual_return'] = backtest_df[TARGET].map({1: 0.001, 0: -0.001})  # Simplified

# Strategy return
backtest_df['strategy_return'] = backtest_df['signal'] * backtest_df['actual_return']

# Cumulative returns
backtest_df['cum_return'] = (1 + backtest_df['strategy_return']).cumprod()
backtest_df['cum_benchmark'] = (1 + backtest_df['actual_return']).cumprod()

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(backtest_df['cum_return'].values, label='Strategy', linewidth=1.5)
ax.plot(backtest_df['cum_benchmark'].values, label='Buy & Hold', linewidth=1.5, alpha=0.7)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Trade #')
ax.set_ylabel('Cumulative Return')
ax.set_title('Strategy vs Buy & Hold (Simplified Backtest)')
ax.legend()
plt.tight_layout()
plt.show()

# Stats
total_trades = len(backtest_df)
winning_trades = (backtest_df['strategy_return'] > 0).sum()
total_return = backtest_df['cum_return'].iloc[-1] - 1

print(f"\nBacktest Summary:")
print(f"  Total trades: {total_trades:,}")
print(f"  Win rate: {winning_trades/total_trades*100:.1f}%")
print(f"  Total return: {total_return*100:.2f}%")

## 8. Next Steps & Research Agenda

### Immediate Actions
1. **Expand dataset**: Load full historical data (multiple days/weeks)
2. **Feature engineering**: Add more lagged features, interaction terms
3. **Hyperparameter tuning**: Increase AutoGluon time_limit for better models
4. **Walk-forward validation**: Implement proper time-series cross-validation

### Research Directions
1. **Multi-asset models**: Train on multiple symbols, look for transfer learning
2. **Regime detection**: Build volatility regime classifier
3. **Ensemble strategies**: Combine predictions from different horizons
4. **Cost analysis**: Add realistic transaction costs and slippage

### Production Considerations
1. Model retraining frequency
2. Feature pipeline latency
3. Position sizing based on prediction confidence
4. Risk management rules

In [ ]:
# ============================================================================
# SAVE RESEARCH ARTIFACTS
# ============================================================================

# Save feature correlation results
corr_df = pd.DataFrame(corr_sorted, columns=['feature', 'correlation_with_ret5tick'])
corr_df.to_csv('research/feature_correlations.csv', index=False)
print("Saved: research/feature_correlations.csv")

# Save model features list
with open('research/model_features.txt', 'w') as f:
    f.write("\n".join(model_features))
print("Saved: research/model_features.txt")

print("\nResearch session complete!")
print(f"Timestamp: {datetime.now()}")